# Oakland 311 calls, part II 
## Prepared by Derek Schwabe
### J233

## Step 1: Importing modules
Just as we did with the Berekeley data for part II, I'm starting by importing pandas and altair for data visualization. I will get the usual warning about lzma, but will not worry about this for now.

In [2]:
import pandas as pd
import altair as alt

/Users/derekschwabe/.pyenv/versions/3.8.5/envs/lecture_06-3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Step 2: Importing the Oakland 311 clean data

I'm going to import the new, cleaned up version of the Oakland 311 dataset that I exported last week called `oakland_311_clean.csv`.

Because a `csv` file is only a plain-text file, it is unable to retain the **dtype** of each column. So I will need to re-designate the `datetime` dtypes correctly again if I want to conduct the same analyses we ran on the Berkeley dataset in class. 

I also want to set **REQUESTID** to an `object` dtype instead of an `int` dtype, because it is simply used as a unique identifier and I don't want to operate on it as if it's a number--I will not be performing any math on it. So it's best to just set it as an object. 

In [3]:
oakland_311 = pd.read_csv('oakland_311_clean.csv', 
    dtype={
        'REQUESTID': object,
    },
    parse_dates=['DATETIMEINIT', 'DATETIMECLOSED', 'CLOSETIME']
)

In [4]:
oakland_311.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
0,1145885,2021-09-09 18:08:56,SeeClickFix,Illegal Dumping (Enforcement Potential),ENVIRON_ENF,"(37.788936670596335, -122.24357499716558)",OPEN,NaN,NaT,6057859.838,2114424.905,CCD2,19X,1438 16TH AVE,Oakland,CA,NaN
1,1146984,2021-09-14 08:08:50,Phone,Illegal Dumping (Enforcement Potential),ENVIRON_ENF,"(37.78829957727266, -122.23117705915959)",OPEN,NaN,NaT,6061437.527,2114126.459,CCD5,21X,2324 E 21ST ST,Oakland,CA,NaN
2,1150718,2021-09-27 07:52:03,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.82142504165362, -122.28732679694832)",OPEN,NaN,NaT,6045445.169,2126491.007,CCD3,07X,ETTIE ST & 28TH ST,Oakland,CA,NaN
3,1151336,2021-09-28 15:37:51,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.75306101818787, -122.16312297802386)",OPEN,NaN,NaT,6080871.846,2100940.775,CCD7,35X,9227 HILLSIDE ST,Oakland,CA,NaN
4,1151681,2021-09-29 16:18:33,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.785503590812, -122.20544605709021)",OPEN,NaN,NaT,6068853.201,2112972.033,CCD4,24Y,3901 ALLENDALE AVE,Oakland,CA,NaN


In [5]:
oakland_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852541 entries, 0 to 852540
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   REQUESTID        852541 non-null  object        
 1   DATETIMEINIT     852541 non-null  datetime64[ns]
 2   SOURCE           852541 non-null  object        
 3   DESCRIPTION      852541 non-null  object        
 4   REQCATEGORY      852460 non-null  object        
 5   REQADDRESS       836654 non-null  object        
 6   STATUS           852540 non-null  object        
 7   REFERREDTO       70168 non-null   object        
 8   DATETIMECLOSED   610875 non-null  datetime64[ns]
 9   SRX              836654 non-null  float64       
 10  SRY              836654 non-null  float64       
 11  COUNCILDISTRICT  809790 non-null  object        
 12  BEAT             812245 non-null  object        
 13  PROBADDRESS      852064 non-null  object        
 14  City             852

Just as occurred with the Berkeley 311 dataset, the **CLOSETIME** column didn't get recorded as `timedelta`. So we will set it using the same function used in class:

In [6]:
oakland_311['CLOSETIME'] = pd.to_timedelta(oakland_311['CLOSETIME']) 

In [7]:
oakland_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852541 entries, 0 to 852540
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   REQUESTID        852541 non-null  object         
 1   DATETIMEINIT     852541 non-null  datetime64[ns] 
 2   SOURCE           852541 non-null  object         
 3   DESCRIPTION      852541 non-null  object         
 4   REQCATEGORY      852460 non-null  object         
 5   REQADDRESS       836654 non-null  object         
 6   STATUS           852540 non-null  object         
 7   REFERREDTO       70168 non-null   object         
 8   DATETIMECLOSED   610875 non-null  datetime64[ns] 
 9   SRX              836654 non-null  float64        
 10  SRY              836654 non-null  float64        
 11  COUNCILDISTRICT  809790 non-null  object         
 12  BEAT             812245 non-null  object         
 13  PROBADDRESS      852064 non-null  object         
 14  City

In [8]:
oakland_311

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
0,1145885,2021-09-09 18:08:56,SeeClickFix,Illegal Dumping (Enforcement Potential),ENVIRON_ENF,"(37.788936670596335, -122.24357499716558)",OPEN,NaN,NaT,6057859.838,2114424.905,CCD2,19X,1438 16TH AVE,Oakland,CA,NaT
1,1146984,2021-09-14 08:08:50,Phone,Illegal Dumping (Enforcement Potential),ENVIRON_ENF,"(37.78829957727266, -122.23117705915959)",OPEN,NaN,NaT,6061437.527,2114126.459,CCD5,21X,2324 E 21ST ST,Oakland,CA,NaT
2,1150718,2021-09-27 07:52:03,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.82142504165362, -122.28732679694832)",OPEN,NaN,NaT,6045445.169,2126491.007,CCD3,07X,ETTIE ST & 28TH ST,Oakland,CA,NaT
3,1151336,2021-09-28 15:37:51,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.75306101818787, -122.16312297802386)",OPEN,NaN,NaT,6080871.846,2100940.775,CCD7,35X,9227 HILLSIDE ST,Oakland,CA,NaT
4,1151681,2021-09-29 16:18:33,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.785503590812, -122.20544605709021)",OPEN,NaN,NaT,6068853.201,2112972.033,CCD4,24Y,3901 ALLENDALE AVE,Oakland,CA,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852536,1151935,2021-09-30 13:54:23,SeeClickFix,Utility - Lid - Missing or Broken,ROW_INSPECTORS,"(37.80086333056584, -122.2728949987278)",OPEN,NaN,NaT,6049471.080,2118926.060,CCD2,03X,988 BROADWAY,Oakland,CA,NaT
852537,1151856,2021-09-30 11:26:13,SeeClickFix,Litter - Street Litter Container - Broken,ILLDUMP,"(37.7990150403045, -122.2729668018932)",OPEN,NaN,NaT,6049437.589,2118253.569,CCD2,03X,7TH ST & FRANKLIN ST,Oakland,CA,NaT
852538,1147185,2021-09-14 14:54:49,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.78429373521214, -122.19783353979535)",OPEN,NaN,NaT,6071044.715,2112491.573,CCD6,28X,3216 COURTLAND AVE,Oakland,CA,NaT
852539,1150481,2021-09-25 15:31:40,SeeClickFix,Homeless Encampment - Living in Vehicle,HE_CLEAN,"(37.75920504225326, -122.18706682059627)",UNFUNDED,NaN,NaT,6073990.955,2103301.471,CCD7,30X,73RD AVE & INTERNATIONAL BLVD,Oakland,CA,NaT


## Step 3: Exploring the data

I'm going to start out by trying to perform some of the same operations on the Oakland 311 data that we performed on the Berkeley 311 data in class to see what kind of results we find for Oakland.

### Looking at the Categories of incidents in 2019

Let's see what kinds of categories of incidents were most common in the year 2019 for Oakland.

I'll start by creating a new dataframe `oakland_311_2019` that limits the `oakland_311` data to just the cases that were open in 2019.

In [11]:
oakland_311_2019 = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2019-01-01') &
    (oakland_311['DATETIMECLOSED'] < '2020-01-01')
].copy()

In [12]:
oakland_311_2019

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
10274,863362,2019-01-03 09:59:00,Other,Parking - Abandoned Vehicle,POLICE,NaN,CLOSED,NaN,2019-01-03 10:24:08,NaN,NaN,NaN,NaN,5224 E 1OTH ST,Oakland,CA,0 days 00:25:08
10275,863684,2019-01-04 10:08:16,Phone,Litter in Parks,PARKS,"(37.84754948342958, -122.25754314069401)",CLOSED,NaN,2019-01-25 14:42:46,6054224.998,2135838.979,CCD1,12Y,COLBY PARK,Oakland,CA,21 days 04:34:30
10278,865383,2019-01-10 12:31:08,Email,Construction and Demolition Recycling,RECYCLING,NaN,CLOSED,NaN,2019-01-10 12:31:37,NaN,NaN,NaN,NaN,ZZ,Oakland,CA,0 days 00:00:29
10279,866189,2019-01-14 10:58:41,Phone,Recycling - Missed Pickup,RECYCLING,NaN,CLOSED,NaN,2019-01-16 09:49:29,NaN,NaN,NaN,NaN,2945 Brookdale Ave,Oakland,CA,1 days 22:50:48
10281,866708,2019-01-16 08:50:56,Phone or Email,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,NaN,CLOSED,NaN,2019-01-28 10:49:30,NaN,NaN,NaN,NaN,60th & Foothill,Oakland,CA,12 days 01:58:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692974,973486,2019-12-31 06:57:37,SeeClickFix,Parking - Enforcement,PARKING,"(37.80346500119062, -122.25095100019593)",CLOSED,NaN,2019-12-31 08:46:40,6055827.757,2119753.865,CCD2,15X,450 STOW AVE,Oakland,CA,0 days 01:49:03
693052,973651,2019-12-31 13:05:13,Phone or Email,Cityworks Config Request - SR Template - Change,CW_DIT_GIS,NaN,CLOSED,NaN,2019-12-31 13:06:29,NaN,NaN,NaN,NaN,SR Template Update OPDAA Submit To Changes,Oakland,CA,0 days 00:01:16
693116,973366,2019-12-30 14:11:38,Phone,Traffic Signal - Outage/Damaged,ELECTRICAL,"(37.83018429811208, -122.27100004964788)",CLOSED,NaN,2019-12-31 07:50:07,6050220.501,2129590.249,CCD1,06X,40TH ST & WEST ST,Oakland,CA,0 days 17:38:29
693255,973127,2019-12-30 08:34:23,Phone or Email,Storm Drains - Other/Complex,DRAINAGE,"(37.83827572727171, -122.23011957195729)",CLOSED,NaN,2019-12-30 16:17:17,6062079.457,2132315.204,CCD1,13X,4815 HARBORD DR,Oakland,CA,0 days 07:42:54


The index of this new dataframe `oakland_311_2019` appears to be no longer sequential. Following what we did in class, I will reset the index to make it sequential. I have two ways to do this that I know of. The first one is to use `df.reset_index(drop=True)`, like this:

```python
oakland_311_2019 = oakland_311_2019.reset_index(drop=True)
```

I can also reset the index at the same time that I subset the data, which I think is easier, so I'm going to do it that way here:

In [14]:
oakland_311_2019 = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2019-01-01') &
    (oakland_311['DATETIMEINIT'] < '2020-01-01')
].reset_index(drop=True)

In [15]:
oakland_311_2019

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
0,937001,2019-08-21 11:11:15,SeeClickFix,Sidewalk - Damage,ROW_STREETSW,"(37.8176670011776, -122.28007049781459)",OPEN,NaN,NaT,6047514.677,2125082.916,CCD3,07X,2521 FILBERT ST,Oakland,CA,NaT
1,895371,2019-04-15 08:30:47,SeeClickFix,Streets - Potholes/Depression,STREETSW,"(37.81366138173259, -122.26229400306427)",OPEN,NaN,NaT,6052621.130,2123527.457,CCD3,08X,258 27TH ST,Oakland,CA,NaT
2,863362,2019-01-03 09:59:00,Other,Parking - Abandoned Vehicle,POLICE,NaN,CLOSED,NaN,2019-01-03 10:24:08,NaN,NaN,NaN,NaN,5224 E 1OTH ST,Oakland,CA,0 days 00:25:08
3,863684,2019-01-04 10:08:16,Phone,Litter in Parks,PARKS,"(37.84754948342958, -122.25754314069401)",CLOSED,NaN,2019-01-25 14:42:46,6054224.998,2135838.979,CCD1,12Y,COLBY PARK,Oakland,CA,21 days 04:34:30
4,864060,2019-01-06 11:26:51,SeeClickFix,Homeless Encampment,HE_CLEAN,"(37.79677419976973, -122.27360019943853)",CANCEL,NaN,NaT,6049239.153,2117441.242,CCD3,01X,381 5TH ST,Oakland,CA,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110923,973498,2019-12-31 08:08:59,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.8393349139426, -122.27625489267614)",CLOSED,NaN,2020-01-22 13:51:29,6048766.377,2132950.380,CCD1,10X,55TH ST & LOWELL ST,Oakland,CA,22 days 05:42:30
110924,973408,2019-12-30 15:40:43,Phone,Tree - Stump Removal,TREES,"(37.798443406459846, -122.1672660767469)",WOCREATE,NaN,NaT,6079968.618,2117484.158,CCD6,25Y,11980 SKYLINE BLVD,Oakland,CA,NaT
110925,973752,2019-12-31 16:32:41,Phone,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.73795224493725, -122.17122891351418)",CLOSED,NaN,2020-01-02 19:14:13,6078430.499,2095482.001,CCD7,32X,104TH AV & ROYAL ANN ST,Oakland,CA,2 days 02:41:32
110926,973276,2019-12-30 11:31:10,Email,Parking - Enforcement,PARKING,"(37.797078713594125, -122.2696860167433)",EVALUATED - NO FURTHER ACTION,NaN,NaT,6050371.998,2117530.694,CCD2,03X,614 ALICE ST,Oakland,CA,NaT


##### Now I'm going to take a look to check out all the unique values of **REQCATEGORY** that exist within the Oakland 311 2019 dataframe.

I will call `series.unique()` on the **REQCATEGORY** column to do this:

In [16]:
oakland_311_2019['REQCATEGORY'].unique()

array(['ROW_STREETSW', 'STREETSW', 'POLICE', 'PARKS', 'HE_CLEAN', 'OTHER',
       'RECYCLING', 'TREES', 'ILLDUMP', 'ELECTRICAL', 'PARKING',
       'ROW_INSPECTORS', 'BLDGMAINT', 'FIRE', 'KOCB', 'DRAINAGE',
       'WATERSHED', 'SEWERS', 'GRAFFITI', 'TRAFFIC', 'TRAFFIC_ENGIN',
       'METER_REPAIR', 'CUT_CLEAN', 'CW_DIT_GIS', nan], dtype=object)

Let's also get a count of all these categories for 2019. I'm going to use `df.groupby()` to do this.

#### Aggregating with `df.groupby()`

I'm including this important note from our lecture for reference, before aggregating by the single **REQCATEGORY** column:

>To aggregate the data, we're going to use a method called `df.groupby()`. Normally, when we group data, we'll group them by columns, like so:

>```python
>df.groupby(['Column 1', 'Column 2'])
>```

>You can also just group by a single column, like we're doing below:

In [17]:
oakland_311_2019.groupby(['REQCATEGORY'])

I also want to inclue this quote from class, to remind myself that `df.group.by()` needs to be accompanied by another method:

>Running a `df.groupby()` doesn't do anything on its own, it just creates a pandas DataFrameGroupBy object. You have to follow it up with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object.

Now I'll use the `df.count()` to do just that here:

In [19]:
oakland_311_2019.groupby(['REQCATEGORY']).count()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
REQCATEGORY,,,,,,,,,,,,,,,,
BLDGMAINT,7638,7638,7638,7638,7584,7638,15,6821,7584,7584,7318,7336,7638,7638,7638,6821
CUT_CLEAN,529,529,529,529,529,529,20,424,529,529,502,503,529,529,529,424
CW_DIT_GIS,28,28,28,28,3,28,0,27,3,3,3,3,28,28,28,27
DRAINAGE,1409,1409,1409,1409,1386,1409,14,1352,1386,1386,1375,1383,1409,1409,1409,1352
ELECTRICAL,2504,2504,2504,2504,2488,2504,1,2236,2488,2488,2370,2378,2504,2504,2504,2236
FIRE,574,574,574,574,571,574,4,100,571,571,568,571,574,574,574,100
GRAFFITI,3918,3918,3918,3918,3899,3918,113,3626,3899,3899,3690,3701,3918,3918,3918,3626
HE_CLEAN,3281,3281,3281,3281,3239,3281,15,150,3239,3239,3220,3225,3281,3281,3281,150
ILLDUMP,33826,33826,33826,33826,33631,33826,72,28914,33631,33631,31485,31671,33817,33826,33826,28914


The "counts" here vary across columns, just as they did for the Berkeley data inclass. To make more sense of this, I'm going to limit the dataframe to a column with zero null cells. I will use **REQUESTID** because I know it has unique values.

I'm also going to go ahead and put the groupby/count results into a new dataframe that I'll name `category_counts_2019`:

In [20]:
category_counts_2019 = oakland_311_2019.groupby(['REQCATEGORY']).count()

I'll also subset the **REQUESTID**, from `category_counts_2019`:

In [21]:
category_counts_2019 = category_counts_2019[['REQUESTID']].copy()
category_counts_2019

,REQUESTID
REQCATEGORY,
BLDGMAINT,7638
CUT_CLEAN,529
CW_DIT_GIS,28
DRAINAGE,1409
ELECTRICAL,2504
FIRE,574
GRAFFITI,3918
HE_CLEAN,3281
ILLDUMP,33826


Now this is looking a lot closer to what I wanted. I now have a simple, clear little dataframe here that lists the names of each cateogry and the number of cases for each. With a quick glance, I can see that the most common category of case in 2019 was for illegal dumping, with 33,826 such cases that year.

#### Resetting the index

As tends to happen when creating new subsets, the index is again no longer a series of sequential integers. To fix that, I will set **REQCATEGORY** to be a column instead of an index.

I'll use `df.reset_index()` to achieve this, using the `drop=True` argument to create a brand new index.

In [22]:
category_counts_2019 = category_counts_2019.reset_index()

In [23]:
category_counts_2019

,REQCATEGORY,REQUESTID
0,BLDGMAINT,7638
1,CUT_CLEAN,529
2,CW_DIT_GIS,28
3,DRAINAGE,1409
4,ELECTRICAL,2504
5,FIRE,574
6,GRAFFITI,3918
7,HE_CLEAN,3281
8,ILLDUMP,33826
9,KOCB,839


#### Renaming the columns

Since the **REQUESTID** column now displays the count of cases for a given category instead of the IDs, I'm going to update the column names to reflect that, re-naming **REQUESTID** to **CASECOUNT**, which keeps with the all-caps formatting style used in the original dataset.

I'm including the following notes from class on the various options to be aware of when changing column names:

>```python
>category_counts_2019.columns = ['Request_Category', 'Case_Count']
>```

>If you have a lot of columns, that's going to be a long list. But if you have a lot of columns to rename, the method above might be easier. If you have only one column to rename out of many columns, you'll want to use the following code:

>```python
>category_counts_2019.rename(columns={'Case_ID': 'Case_Count'}, inplace=True)
>```

>The first argument in the `df.rename()` method is `columns`. And what do we set columns to? We set it to a Python dictionary where the "key" is the original column name and the "value" is the new column name: `{'Case_ID': 'Case_Count'}`. 

>The second argument is `inplace=True`. That tells us to change the `category_counts_2019` "in place" or without having to reset the dataframe variable. A lot of the methods in pandas return a new dataframe instead of altering the original dataframe. An alternative to using `inplace` is the following code:

>```python
>category_counts_2019 = category_counts_2019.rename(columns={'Case_ID': 'Count'})
>```

In [32]:
category_counts_2019.rename(columns={'REQUESTID': 'CASECOUNT'}, inplace=True)

And now I'll print the table to see if the re-naming worked:

In [33]:
category_counts_2019

,REQCATEGORY,CASECOUNT
0,BLDGMAINT,7638
1,CUT_CLEAN,529
2,CW_DIT_GIS,28
3,DRAINAGE,1409
4,ELECTRICAL,2504
5,FIRE,574
6,GRAFFITI,3918
7,HE_CLEAN,3281
8,ILLDUMP,33826
9,KOCB,839


Now, I'll sort the dataframe by **CASECOUNT** to make it easier to see which categories were most and least common:

In [34]:
category_counts_2019 = category_counts_2019.sort_values(by=['CASECOUNT'], ascending=False).copy()
category_counts_2019

,REQCATEGORY,CASECOUNT
8,ILLDUMP,33826
14,POLICE,22773
11,OTHER,8016
0,BLDGMAINT,7638
19,STREETSW,7064
6,GRAFFITI,3918
7,HE_CLEAN,3281
12,PARKING,3145
22,TREES,2761
4,ELECTRICAL,2504


Now it's even easier to see that Illegal Dumping and Police calls were the most common categories of 411 cases for Oakland in 2019. 

Once again the index is out of order. I'll use again `df.reset_index(drop=True)` to get that back in order, just to keep things tidy. 

In [35]:
category_counts_2019 = category_counts_2019.reset_index(drop=True)
category_counts_2019

,REQCATEGORY,CASECOUNT
0,ILLDUMP,33826
1,POLICE,22773
2,OTHER,8016
3,BLDGMAINT,7638
4,STREETSW,7064
5,GRAFFITI,3918
6,HE_CLEAN,3281
7,PARKING,3145
8,TREES,2761
9,ELECTRICAL,2504


#### It's time to visualize this brand new table!

Before I run the Altair code below, I'm going to review the notes from class on how to use it, drawing on the Altair documentation:

>```python
>alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y='Request_Category'
).properties(
    title='Berkeley 311 cases in 2019'
)
>```
>The first part of the code `alt.Chart()` requires you to fill the first argument with a dataframe, in this case `category_counts_2019`.

>The next part of the code `mark_bar()` specifies a bar chart. (If you want a line chart, you'd use `mark_line()`.)

>After that, `.encode()` tells Altair which columns to use for the `x` and `y` axes.

>If you want to add a title, you'd use Altair's `.properties()` method.

In [39]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='CASECOUNT',
    y='REQCATEGORY'
).properties(
    title='Oakland 311 cases in 2019 by category case count'
)

alt.Chart(...)

Looking good, but I want to sort the chart in descending order. Again following the below notes from class based on the Berkeley data, I can follow the same steps to create a custom Y encoding:

>```python
>alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y=alt.Y('Request_Category', sort='-x')
).properties(
    title='Berkeley 311 cases in 2019'
)
>```

>Basically, you have to create a custom Y encoding with the format: `alt.Y('column_name', sort='-x')`. `-x` means the inverse of the x-axis, in this case. This is not intuitive, I think — it's just something you'd have to look up in the documentation.

In [38]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='CASECOUNT',
    y=alt.Y('REQCATEGORY', sort='-x')
).properties(
    title='Oakland 311 cases in 2019 by category case count'
)

alt.Chart(...)

### Counting total incidents per year

Next I'm going to get a count of all the incidents that were reported to Oakland 311 by year. Since the data for 2009 and 2021 are not complete, I will need to do some subsetting to remove those years and only include years for which I have full data:

Ill start by creating a new dataframe called `oakland_311_complete` that limits the `oakland_311` dataframe to ones in which the **DATETIMEINIT** value starts on or after January 1, 2010 and is before January 1, 2021. 

In [40]:
oakland_311_complete = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2010-01-01') &
    (oakland_311['DATETIMEINIT'] < '2021-01-01')
].reset_index(drop=True)

#### Use df.groupby() with datetimes

Drawing on what I learned from class on grouping datetimes, I'm going to use  `pd.Grouper`. Here's the class overview of the various arguments within it, which is useful information to have in front of us:

>Before we run the code below, let's look at the different arguments within the method:

>```python
pd.Grouper(key='Date_Opened', axis=0, freq='A')
>```

>The `key` argument lists the column. The `axis` argument is `0`. In pandas, axis 0 is rows and axis 1 means columns. That means you can do column-wise calculations if your data is shaped differently. 

>The `freq` argument is `A`, which stands for "annual" or year (`Y` also works, but isn't documented). You can see other [frequency arguments](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) in the official pandas documentation.

In [41]:
oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A')])

Once again I have to chain the `.groupby()` command with another method to create a new dataframe. I'll use `df.count()` on the DataFrameGroupBy object and will name the new dataframe `oakland_annual_cases`.

In [42]:
oakland_annual_cases = oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A')]).count()

In [43]:
oakland_annual_cases

,REQUESTID,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
DATETIMEINIT,,,,,,,,,,,,,,,,
2010-12-31,33647,33647,33647,33646,32622,33647,7,31504,32622,32622,30666,30566,33540,33647,33647,31504
2011-12-31,37995,37995,37995,37988,36988,37995,414,33952,36988,36988,34962,35021,37920,37995,37995,33952
2012-12-31,47294,47294,47294,47287,46233,47294,4776,36695,46233,46233,43922,44103,47220,47294,47294,36695
2013-12-31,56888,56888,56888,56884,55004,56888,6632,40366,55004,55004,51317,51416,56871,56888,56888,40366
2014-12-31,61826,61826,61826,61826,60769,61826,7395,45511,60769,60769,59102,59267,61799,61826,61826,45511
2015-12-31,66889,66889,66889,66869,65588,66889,7526,49238,65588,65588,64097,64316,66867,66889,66889,49238
2016-12-31,75932,75932,75932,75926,74372,75932,7681,54163,74372,74372,72466,72637,75908,75932,75932,54163
2017-12-31,80740,80740,80740,80740,79446,80740,8548,53035,79446,79446,76747,76967,80729,80740,80740,53035
2018-12-31,77851,77851,77851,77843,77066,77851,10313,50150,77066,77066,76193,76417,77829,77851,77851,50150


Now I'll subset **REQUESTID**, from annual cases, then reset the index so that `DATETIMEINIT` becomes a new column:

In [45]:
oakland_annual_cases = oakland_annual_cases[['REQUESTID']].reset_index()
oakland_annual_cases

,DATETIMEINIT,REQUESTID
0,2010-12-31,33647
1,2011-12-31,37995
2,2012-12-31,47294
3,2013-12-31,56888
4,2014-12-31,61826
5,2015-12-31,66889
6,2016-12-31,75932
7,2017-12-31,80740
8,2018-12-31,77851
9,2019-12-31,110928


#### Renaming columns
Once again I need to re-name the **REQUESTID** column to reflect the count that it is now representing.

In [49]:
oakland_annual_cases.rename(columns={'REQUESTID': 'CASECOUNT'}, inplace=True)

Here is the new summary table with properly named columns:

In [50]:
oakland_annual_cases

,DATETIMEINIT,CASECOUNT
0,2010-12-31,33647
1,2011-12-31,37995
2,2012-12-31,47294
3,2013-12-31,56888
4,2014-12-31,61826
5,2015-12-31,66889
6,2016-12-31,75932
7,2017-12-31,80740
8,2018-12-31,77851
9,2019-12-31,110928


I dont love the name of the year column heading as it doesn't reflect the information as well. So I'll create a new, simpler column in `oakland_annual_cases` and call it **YEAR**:

In [52]:
oakland_annual_cases['YEAR'] = oakland_annual_cases['DATETIMEINIT'].dt.year

In [53]:
oakland_annual_cases

,DATETIMEINIT,CASECOUNT,YEAR
0,2010-12-31,33647,2010
1,2011-12-31,37995,2011
2,2012-12-31,47294,2012
3,2013-12-31,56888,2013
4,2014-12-31,61826,2014
5,2015-12-31,66889,2015
6,2016-12-31,75932,2016
7,2017-12-31,80740,2017
8,2018-12-31,77851,2018
9,2019-12-31,110928,2019


Since I don't really need the **DATETIMEINIT** column anymore and it is confusing, I'll remove it by subsetting the dataframe with just the other two columns:

In [54]:
oakland_annual_cases = oakland_annual_cases[['YEAR', 'CASECOUNT']].copy()

In [55]:
oakland_annual_cases

,YEAR,CASECOUNT
0,2010,33647
1,2011,37995
2,2012,47294
3,2013,56888
4,2014,61826
5,2015,66889
6,2016,75932
7,2017,80740
8,2018,77851
9,2019,110928


#### Time to Visualize this Annual Casecount Data

In [56]:
alt.Chart(oakland_annual_cases).mark_bar().encode(
    x='YEAR',
    y='CASECOUNT'
)

alt.Chart(...)

Looking good, though the **YEAR** is formatted as a number, including a comma. I'm going to try to fix this using the same alteration we used on the Berkeley dataset. Here's the helpful note from class pointing out what's going on here in the documentation that allows me to remove that annoying comma (it also will allow me to add a title!):

>Before you run the code below, notice that after `Year` there's a colon and an `O`. The `O` is shorthand for "ordinal," and tells Altair to treat `Year` as if it's a discrete quantity (a.k.a. integers), not a continuous quantity (e.g. a number with decimals). 

In [57]:
alt.Chart(oakland_annual_cases).mark_bar().encode(
    x='YEAR:O',
    y='CASECOUNT'
).properties(
    title='Oakland 311 calls: Total number of cases by year'
)

alt.Chart(...)

I'm keeping this link here on [Altair encoding types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types) in the Altair documentation, as it will be an extremely helpful reference moving forward.

### Finding the Median CLOSETIME by year

My next analysis tas will be to find the median length of time it takes to close a case by year. 

Building on what we learned in lecture, I'll need to start by creating a new column called **CLOSETIMESECONDS** that converts **CLOSETIME** to seconds. Then I can calculate the median number of seconds.

In [63]:
oakland_311_complete['CLOSETIMESECONDS'] = oakland_311_complete['CLOSETIME'].dt.total_seconds()

I'll use df.head() to see how this worked out:

In [59]:
oakland_311_complete.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME,CLOSETIMESECONDS
0,369975,2011-05-12 14:43:40,Phone or Email,Sidewalk - Damage,ROW_STREETSW,"(37.796927666729566, -122.1983845122619)",OPEN,NaN,NaT,6070969.000,2117094.000,CCD4,22Y,3908 NORTON AV,Oakland,CA,NaT,NaN
1,1060771,2020-10-28 11:40:01,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.75078101633127, -122.17064104205342)",CLOSED,NaN,2021-10-02 12:21:22,6078683.848,2100149.425,CCD7,34X,9016 PLYMOUTH ST,Oakland,CA,339 days 00:41:21,29292081.0
2,1052966,2020-10-03 09:37:44,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.747658843338996, -122.172764281517)",CLOSED,NaN,2021-10-02 10:46:00,6078049.765,2099023.727,CCD7,34X,9302 INTERNATIONAL BLVD,Oakland,CA,364 days 01:08:16,31453696.0
3,1064670,2020-11-11 17:09:47,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.75601000381378, -122.1746169446731)",CLOSED,NaN,2021-10-02 12:11:15,6077568.604,2102073.642,CCD6,34X,84TH AVE & PLYMOUTH ST,Oakland,CA,324 days 19:01:28,28062088.0
4,364974,2011-03-28 10:03:59,Phone or Email,Streets/Sidewalks - Unsafe Condition,STREETSW,"(37.76677228662963, -122.18717970372998)",OPEN,NaN,NaT,6074008.000,2106057.000,CCD6,30X,2445 66TH AV,Oakland,CA,NaT,NaN


Now I can run the groupby/median code we used in class to find that median close time:

In [60]:
median_close_time = oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A'), ]).median()

In [61]:
median_close_time

,SRX,SRY,CLOSETIMESECONDS
DATETIMEINIT,,,
2010-12-31,6.060859e+06,2.117159e+06,271677.0
2011-12-31,6.060215e+06,2.117932e+06,345433.0
2012-12-31,6.059595e+06,2.118648e+06,440519.0
2013-12-31,6.058980e+06,2.118543e+06,520046.0
2014-12-31,6.056907e+06,2.119122e+06,429433.0
2015-12-31,6.056996e+06,2.118844e+06,448167.5
2016-12-31,6.060108e+06,2.117722e+06,519751.0
2017-12-31,6.061456e+06,2.117413e+06,359105.0
2018-12-31,6.060343e+06,2.117892e+06,354149.5


Looks like it worked! Now I'll subset for just the **DATETIMEINIT** and **CLOSETIMESECONDS** columns, and set **DATETIMEINIT** to its own column.

In [62]:
median_close_time = median_close_time[['CLOSETIMESECONDS']].reset_index()
median_close_time

,DATETIMEINIT,CLOSETIMESECONDS
0,2010-12-31,271677.0
1,2011-12-31,345433.0
2,2012-12-31,440519.0
3,2013-12-31,520046.0
4,2014-12-31,429433.0
5,2015-12-31,448167.5
6,2016-12-31,519751.0
7,2017-12-31,359105.0
8,2018-12-31,354149.5
9,2019-12-31,516791.5


Now that I have the basics in place, I can continue to clean up this table. I'll now create new column called **YEAR**, as we did before, and convert the seconds to days using some simple arethmatic to make it easier to decypher that column:

In [64]:
median_close_time['YEAR'] = median_close_time['DATETIMEINIT'].dt.year
median_close_time['CLOSETIMEDAYS'] = median_close_time['CLOSETIMESECONDS'] / 60 / 60 / 24

It's a good time to take another look at this dataframe before I subset it to remove the columns I no longer need:

In [65]:
median_close_time

,DATETIMEINIT,CLOSETIMESECONDS,YEAR,CLOSETIMEDAYS
0,2010-12-31,271677.0,2010,3.144410
1,2011-12-31,345433.0,2011,3.998067
2,2012-12-31,440519.0,2012,5.098600
3,2013-12-31,520046.0,2013,6.019051
4,2014-12-31,429433.0,2014,4.970289
5,2015-12-31,448167.5,2015,5.187124
6,2016-12-31,519751.0,2016,6.015637
7,2017-12-31,359105.0,2017,4.156308
8,2018-12-31,354149.5,2018,4.098953
9,2019-12-31,516791.5,2019,5.981383


In [66]:
median_close_time = median_close_time[['YEAR', 'CLOSETIMEDAYS']].copy()

In [67]:
median_close_time

,YEAR,CLOSETIMEDAYS
0,2010,3.144410
1,2011,3.998067
2,2012,5.098600
3,2013,6.019051
4,2014,4.970289
5,2015,5.187124
6,2016,6.015637
7,2017,4.156308
8,2018,4.098953
9,2019,5.981383


This is looking much better! Time to visualize it. 
### Visualizing the Median Close Time Data by Year for Oakland 311

In [68]:
alt.Chart(median_close_time[['YEAR', 'CLOSETIMEDAYS']]).mark_bar().encode(
    x='YEAR:O',
    y='CLOSETIMEDAYS',
).properties(
    title='Oakland 311 calls: Median resolution time in days for each year'
)

alt.Chart(...)

It looks as though the median case resolution time has been quite erratic over the last decade or so. It almost appears as if it rises and falls in waves. This is an interesting pattern that might be work looking into...

### Merging two dataframes

Now I'd like to merge `median_close_time` and `oakland_annual_cases` in order get some more practice with merging dataframes! I'll start by taking a look at the two dataframes that I'm merging once more:

In [69]:
oakland_annual_cases

,YEAR,CASECOUNT
0,2010,33647
1,2011,37995
2,2012,47294
3,2013,56888
4,2014,61826
5,2015,66889
6,2016,75932
7,2017,80740
8,2018,77851
9,2019,110928


In [70]:
median_close_time

,YEAR,CLOSETIMEDAYS
0,2010,3.144410
1,2011,3.998067
2,2012,5.098600
3,2013,6.019051
4,2014,4.970289
5,2015,5.187124
6,2016,6.015637
7,2017,4.156308
8,2018,4.098953
9,2019,5.981383


Once again I'm keeping the notes from class here handy, which break down the arguments for the `pd.merge()` function we'll be using:

>Let's look at the arguments in `pd.merge()` before we run it:

>```python
>pd.merge(
    df1,
    df2,
    how='outer', # other options: 'inner', 'left', 'right'
    on='Year',
    validate='1:1' # options: '1:m', 'm:m', 'm:1'
>)
>```
>1. The first argument is the left-hand dataframe. The second argument is the right-hand dataframe. Why is it important that there's an order to dataframes? 

>2. The `how` argument tells pandas how we'll merge the two dataframes. In this case, we'll use `outer`. But we could also use `left`, `right`, or `inner`. What does this mean? Let's take a look at the lecture slides to view a visual. In this case, it doesn't matter which value we use for `how` because both dataframes have 10 rows with matching years.

>3. The `on` argument tells pandas which column key we're going to match on. In this case, we want the years to match up.

>4. The `validate` argument is optional, but I recommend you learn how to use it. The value we used, `'1:1'` means that 1 row in the left-hand dataframe will match up to exactly 1 row in the right-hand dataframe. The option `1:m` means that 1 row in the left-hand dataframe could match up to **many** rows in the right-hand dataframe. (Any time you use `m`, you're telling pandas that there _might_ be multiple matches.)

Using this, I'll set up the `oakland_annual_summary` merged dataframe, joining the two on **YEAR**:


In [71]:
oakland_annual_summary = pd.merge(
    oakland_annual_cases,
    median_close_time,
    on='YEAR',
    how='outer',
    validate='1:1'
)

In [72]:
oakland_annual_summary

,YEAR,CASECOUNT,CLOSETIMEDAYS
0,2010,33647,3.144410
1,2011,37995,3.998067
2,2012,47294,5.098600
3,2013,56888,6.019051
4,2014,61826,4.970289
5,2015,66889,5.187124
6,2016,75932,6.015637
7,2017,80740,4.156308
8,2018,77851,4.098953
9,2019,110928,5.981383


## Step 3: Trying Some New Analysis

Now that I've had the chance to replicate the steps we took in class with the Oakland 311 data, I'm going to see if I can apply what I've learned to do some additional analysis on the dataset. 

## Finding most popular ways of submitting 311 complaints in 2010 and 2019
I noticed when looking over the data using `df.info()` that there is a column called **SOURCE**. The website links describes this as the method used to contact 411. I'll start out using `series.unique()` to see how many options exist for this:

In [86]:
oakland_311['SOURCE'].unique()

array(['SeeClickFix', 'Phone', 'Other', 'Email', 'Phone or Email',
       'Voicemail', 'Website', 'City Attorney'], dtype=object)

So there are 8 total categories and most of them are pretty straightforward. 'Phone or Email' means the informat likely used both, while SeeClickFix is a company hired by Oakland to create and run a website and app for reporting incidents. It's still an online process, but slightly different than just reporting directly through the city website. 

#### Step 1: Creating a dataframe for the count of cases by source in 2019
First I'll use `.groupby()` and `df.count()` to find out how many incidents in 2019 were reported in which source and create a dataframe to display it:

In [148]:
oakland_311_2019.groupby(['SOURCE'])

In [149]:
oakland_311_2019.groupby(['SOURCE']).count()

,REQUESTID,DATETIMEINIT,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,CLOSETIME
SOURCE,,,,,,,,,,,,,,,,
City Attorney,3,3,3,3,3,3,0,2,3,3,3,3,3,3,3,2
Email,8753,8753,8753,8753,8559,8753,996,6509,8559,8559,8523,8544,8753,8753,8753,6509
Other,765,765,765,765,719,765,96,495,719,719,715,717,765,765,765,495
Phone,21668,21668,21668,21668,21396,21668,1779,16302,21396,21396,21322,21370,21668,21668,21668,16302
Phone or Email,3632,3632,3632,3628,3322,3632,68,3170,3322,3322,3300,3364,3624,3632,3632,3170
SeeClickFix,73980,73980,73980,73967,73957,73980,3692,49265,73957,73957,68675,68963,73959,73980,73980,49265
Voicemail,2086,2086,2086,2086,1772,2086,121,1736,1772,1772,1760,1769,2086,2086,2086,1736
Website,41,41,41,41,35,41,0,41,35,35,35,35,41,41,41,41


In [150]:
source_counts_2019 = oakland_311_2019.groupby(['SOURCE']).count()

Now I'll isolute just the column I'm interested in (**SOURCE**) and create a new dataframe that just includes that column and the number of cases for each:

In [151]:
source_counts_2019 = source_counts_2019[['REQUESTID']].copy()
source_counts_2019

,REQUESTID
SOURCE,
City Attorney,3
Email,8753
Other,765
Phone,21668
Phone or Email,3632
SeeClickFix,73980
Voicemail,2086
Website,41


Here I'll do some re-naming to make the columns match what they represent:

In [152]:
source_counts_2019.rename(columns={'REQUESTID': 'CASECOUNT2019'}, inplace=True)

In [153]:
source_counts_2019

,CASECOUNT2019
SOURCE,
City Attorney,3
Email,8753
Other,765
Phone,21668
Phone or Email,3632
SeeClickFix,73980
Voicemail,2086
Website,41


Now I'll sort the sources from most cases to least cases:

In [155]:
source_counts_2019 = source_counts_2019.sort_values(by=['CASECOUNT2019'], ascending=False).copy()
source_counts_2019

,CASECOUNT2019
SOURCE,
SeeClickFix,73980
Phone,21668
Email,8753
Phone or Email,3632
Voicemail,2086
Other,765
Website,41
City Attorney,3


And here I'll make source a column and reset the index:

In [156]:
source_counts_2019 = source_counts_2019.reset_index()
source_counts_2019

,SOURCE,CASECOUNT2019
0,SeeClickFix,73980
1,Phone,21668
2,Email,8753
3,Phone or Email,3632
4,Voicemail,2086
5,Other,765
6,Website,41
7,City Attorney,3


#### Step 2: Repeating the same operations for 2010 data 

##### Creating a dataframe for just Oakland 311 cases in 2010

In [157]:
oakland_311_2010 = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2010-01-01') &
    (oakland_311['DATETIMEINIT'] < '2011-01-01')
].reset_index(drop=True)

##### Checking out the new 2010 cases dataframe:

In [158]:
oakland_311_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33647 entries, 0 to 33646
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   REQUESTID        33647 non-null  object         
 1   DATETIMEINIT     33647 non-null  datetime64[ns] 
 2   SOURCE           33647 non-null  object         
 3   DESCRIPTION      33647 non-null  object         
 4   REQCATEGORY      33646 non-null  object         
 5   REQADDRESS       32622 non-null  object         
 6   STATUS           33647 non-null  object         
 7   REFERREDTO       7 non-null      object         
 8   DATETIMECLOSED   31504 non-null  datetime64[ns] 
 9   SRX              32622 non-null  float64        
 10  SRY              32622 non-null  float64        
 11  COUNCILDISTRICT  30666 non-null  object         
 12  BEAT             30566 non-null  object         
 13  PROBADDRESS      33540 non-null  object         
 14  City             33647

##### Using `df.groupby()` and `df.count()` to create a new dataframe grouped by source type:

In [159]:
oakland_311_2010.groupby(['SOURCE'])

In [160]:
source_counts_2010 = oakland_311_2010.groupby(['SOURCE']).count()

##### Isolating just the columns I want (the Source name and the request ID):

In [161]:
source_counts_2010 = source_counts_2010[['REQUESTID']].copy()
source_counts_2010

,REQUESTID
SOURCE,
Phone or Email,31077
Website,2570


##### Renaming the columns and re-setting the index:

In [162]:
source_counts_2010.rename(columns={'REQUESTID': 'CASECOUNT2010'}, inplace=True)


##### Sorting the categories from most case counts to least and re-setting the index:

In [163]:
source_counts_2010 = source_counts_2010.sort_values(by=['CASECOUNT2010'], ascending=False).copy()
source_counts_2010 = source_counts_2010.reset_index()
source_counts_2010


,SOURCE,CASECOUNT2010
0,Phone or Email,31077
1,Website,2570


#### Step 3: Visualizing these two new tables with Altair

In [166]:
alt.Chart(source_counts_2019).mark_bar().encode(
    x='CASECOUNT2019',
    y=alt.Y('SOURCE', sort='-x')
).properties(
    title='Oakland 311 cases in 2019 by source and case count'
)

alt.Chart(...)

In [167]:
alt.Chart(source_counts_2010).mark_bar().encode(
    x='CASECOUNT2010',
    y=alt.Y('SOURCE', sort='-x')
).properties(
    title='Oakland 311 cases in 2010 by source and case count'
)

alt.Chart(...)

As the above visualizations show, it seems that there was a major expansion of either options offered for reporting 311 calls between 2010 and 2019, or the city of Oakland started collecting more granular data on source. This could be worth looking into.

#### Step 4: Merging the two dataframes to create a summary table

Building on what I've learned, I'm going to use `pd.merge()` to bring these two tables together. I'll use an outer join to make sure I capture all the categories used across both years--not just the ones the two years have in common:

In [168]:
oakland_case_summary_2010_and_2019 = pd.merge(
    source_counts_2010,
    source_counts_2019,
    on='SOURCE',
    how='outer',
    validate='1:1'
)

Now I'll print the new summary table and have a look!

In [169]:
oakland_case_summary_2010_and_2019

,SOURCE,CASECOUNT2010,CASECOUNT2019
0,Phone or Email,31077.0,3632
1,Website,2570.0,41
2,SeeClickFix,NaN,73980
3,Phone,NaN,21668
4,Email,NaN,8753
5,Voicemail,NaN,2086
6,Other,NaN,765
7,City Attorney,NaN,3


This new summary table allows me to more easily compare source numbers between the two years. A quick glance at this tells me that there was a significant winding down of phone and email reports and a signficiant rise in SeeClickFix reports between 2010-2019. There has also been a significant rise in overall cases reported in this time. This makes me more curious to know about this new SeeClickFix service and how it works. Another possible next step would be to run these steps for all years in order to get a better look at the progression. 